### Section 1.4.1 Dividers
Taken from "Switch-Mode Power Supplies" by Christophe P. Basso

In [1]:
# Imports, type aliases

import copy
import tomllib
from pathlib import Path
from typing import Any

import numpy as np
import numpy.typing as npt
import py4spice as spi

# Aliases for type checking
from py4spice.globals_types import numpy_flt

In [2]:
# Constants

CONFIG_FILENAME: Path = Path("./config.toml")
PROJECT_SECTION_CHECK: str = "SEC_1_04_01"


class Key:
    """Keys for dictionaries.  Defined here at top level so they can be
    referenced instead of using strings for keys.
    """

    # Keys for decoding config file
    GLOBAL: str = "GLOBAL"
    PROJECT: str = "PROJECT"
    NGSPICE_EXE_STR: str = "NGSPICE_EXE_STR"
    NETLISTS_DIR_STR: str = "NETLISTS_DIR_STR"
    RESULTS_DIR_STR: str = "RESULTS_DIR_STR"
    SIM_TRANSCRIPT_STR: str = "SIM_TRANSCRIPT_STR"
    PROJ_PATH_STR: str = "PROJ_PATH_STR"

    # Keys for the paths_dict
    NGSPICE_EXE = "ngspice_exe"
    PROJ_PATH = "proj_path"
    NETLISTS_PATH = "netlists_path"
    RESULTS_PATH = "results_path"
    SIM_TRANSCRIPT_FILENAME = "sim_transcript_filename"

    # Keys for the netlists_dict
    BLANKLINE = "blankline"
    TITLE = "title"
    END_LINE = "end_line"
    STIMULUS = "stimulus"
    SUPPLIES = "supplies"
    DUT = "dut"
    CONTROL = "control"
    TOP1 = "top1"

    # Keys for the vectors_dict
    VEC_ALL = "vec_all"
    VEC_ALL_EXPANDED = "vec_all_expanded"
    VEC_VIN = "vec_vin"
    VEC_IIN = "vec_iin"
    VEC_VOUT1 = "vec_vout1"
    VEC_VOUT2 = "vec_vout2"
    VEC_IOUT1 = "vec_iout1"
    VEC_IOUT2 = "vec_iout2"
    VEC_IN_OUT = "vec_in_out"

In [3]:
# Initialization


def initialize() -> (
    tuple[dict[str, Path], dict[str, spi.Netlist], dict[str, spi.Vectors]]
):
    """All the setup we can do before running the different parts of the project"""
    # read config file and create config dictionary
    with open(CONFIG_FILENAME, "rb") as file:
        my_config: dict[str, Any] = tomllib.load(file)

    # key for project section in config file
    proj_section_key: str = my_config["GLOBAL"]["PROJECT"] 
    
    # check if project_section_key is equal to PROJECT_SECTION_CHECK
    if proj_section_key != PROJECT_SECTION_CHECK:
        raise ValueError(f"Error: project_section_key is not equal to {PROJECT_SECTION_CHECK}. Fix {CONFIG_FILENAME} so PROJECT = {PROJECT_SECTION_CHECK}")
    
    # Continue, and create project path
    proj_path: Path = Path(my_config[proj_section_key][Key.PROJ_PATH_STR])

    # Create paths based on the config dictionary
    ngspice_exe: Path = Path(my_config[Key.GLOBAL][Key.NGSPICE_EXE_STR])
    netlists_path: Path = proj_path / my_config[Key.GLOBAL][Key.NETLISTS_DIR_STR]
    results_path: Path = proj_path / my_config[Key.GLOBAL][Key.RESULTS_DIR_STR]

    # create results directory if it does not exist
    results_path.mkdir(parents=True, exist_ok=True)

    # create simlulation transcript file. If it exists, make sure it is empty
    sim_tran_filename: Path = (
        results_path / my_config[Key.GLOBAL][Key.SIM_TRANSCRIPT_STR]
    )
    if sim_tran_filename.exists():  # delete and recreate. this makes sure it's empty
        sim_tran_filename.unlink()
    sim_tran_filename.touch()

    # create paths dictionary
    paths_dict = {
        Key.NGSPICE_EXE: ngspice_exe,
        Key.PROJ_PATH: proj_path,
        Key.NETLISTS_PATH: netlists_path,
        Key.RESULTS_PATH: results_path,
        Key.SIM_TRANSCRIPT_FILENAME: sim_tran_filename,
    }

    # netlists_dict = define_netlists(paths_dict)
    netlists_path: Path = paths_dict[Key.NETLISTS_PATH]  # make shorter alias
    netlists_dict: dict[str, spi.Netlist] = {}  # create empty netlist dictionary

    netlists_dict[Key.BLANKLINE] = spi.Netlist("")  # blank line for spacing
    netlists_dict[Key.TITLE] = spi.Netlist("* Title line")  # title line
    netlists_dict[Key.END_LINE] = spi.Netlist(".end")  # end statement

    # create netlist objects from files and add to netlist dictionary
    netlists_dict[Key.DUT] = spi.Netlist(netlists_path / "dut.cir")
    netlists_dict[Key.STIMULUS] = spi.Netlist(netlists_path / "stimulus.cir")
    netlists_dict[Key.SUPPLIES] = spi.Netlist(netlists_path / "supplies.cir")

    # Define a vector dictionary for simulation and post-simulation analysis
    vectors_dict = {
        Key.VEC_ALL: spi.Vectors("all"),
        Key.VEC_ALL_EXPANDED: spi.Vectors(
            "in out1 out1_meas out2 out2_meas vee vee#branch vin#branch vmeas1#branch vmeas2#branch"
        ),
        Key.VEC_VIN: spi.Vectors("in"),
        Key.VEC_IIN: spi.Vectors("vin#branch"),
        Key.VEC_VOUT1: spi.Vectors("out1"),
        Key.VEC_VOUT2: spi.Vectors("out2"),
        Key.VEC_IOUT1: spi.Vectors("vmeas1#branch"),
        Key.VEC_IOUT2: spi.Vectors("vmeas2#branch"),
    }
    # create this vector as a combination of the other vectors
    vectors_dict[Key.VEC_IN_OUT] = (
        vectors_dict[Key.VEC_VIN]
        + vectors_dict[Key.VEC_VOUT1]
        + vectors_dict[Key.VEC_VOUT2]
    )
    return paths_dict, netlists_dict, vectors_dict


paths_dict, netlists_dict, vectors_dict = initialize()

#### Part 1: operating point and dc transfer
Say something!

In [4]:
# Part 1


def part1(
    my_paths_dict: dict[str, Path],
    my_netlists_dict: dict[str, spi.Netlist],
    my_vectors_dict: dict[str, spi.Vectors],
) -> dict[str, spi.Netlist]:

    # Define analyses
    list_of_analyses: list[spi.Analyses] = []  # start with an empty list

    # 1st analysis: operating point
    op1 = spi.Analyses(
        name="op1",
        cmd_type="op",
        cmd="op",
        vector=my_vectors_dict[Key.VEC_ALL],
        results_loc=my_paths_dict[Key.RESULTS_PATH],
    )
    list_of_analyses.append(op1)

    # create control section
    my_control = spi.Control()  # create 'my_control' object
    for analysis in list_of_analyses:
        my_control.insert_lines(analysis.lines_for_cntl())
    my_netlists_dict[Key.CONTROL] = spi.Netlist(str(my_control))

    # concatenate all tne netlists to make top1 and add to netlist dict
    my_netlists_dict[Key.TOP1] = (
        my_netlists_dict[Key.TITLE]
        + my_netlists_dict[Key.BLANKLINE]
        + my_netlists_dict[Key.DUT]
        + my_netlists_dict[Key.BLANKLINE]
        + my_netlists_dict[Key.SUPPLIES]
        + my_netlists_dict[Key.BLANKLINE]
        + my_netlists_dict[Key.STIMULUS]
        + my_netlists_dict[Key.BLANKLINE]
        + my_netlists_dict[Key.BLANKLINE]
        + my_netlists_dict[Key.CONTROL]
        + my_netlists_dict[Key.END_LINE]
        + my_netlists_dict[Key.BLANKLINE]
    )
    # write netlist to a file so ngspice can read it
    top_filename: Path = my_paths_dict[Key.NETLISTS_PATH] / "top1.cir"
    my_netlists_dict[Key.TOP1].write_to_file(top_filename)

    # prepare simulate object, print out command, and simulate
    sim: spi.Simulate = spi.Simulate(
        ngspice_exe=my_paths_dict[Key.NGSPICE_EXE],
        netlist_filename=top_filename,
        transcript_filename=my_paths_dict[Key.SIM_TRANSCRIPT_FILENAME],
        name="sim1",
        timeout=20,
    )
    # spi.print_section("Ngspice Command", sim1) # print out command
    sim.run()  # run the Ngspice simulation

    # convert the raw results into list of SimResults objects
    sim_results: list[spi.SimResults] = [
        spi.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
        for analysis in list_of_analyses
    ]
    (my_op1,) = sim_results  # give each SimResults object an easier name

    # diaplay results for operating point analysis
    spi.print_section("Operating Point Results", my_op1.table_for_print())

    # Calculate power and efficiency
    vin: float = my_op1.data_table[str(vectors_dict[Key.VEC_VIN])]
    iin: float = my_op1.data_table[str(vectors_dict[Key.VEC_IIN])]
    vout1: float = my_op1.data_table[str(vectors_dict[Key.VEC_VOUT1])]
    vout2: float = my_op1.data_table[str(vectors_dict[Key.VEC_VOUT2])]
    iout1: float = my_op1.data_table[str(vectors_dict[Key.VEC_IOUT1])]
    iout2: float = my_op1.data_table[str(vectors_dict[Key.VEC_IOUT2])]

    pr1: float = (vin - vout1) * iout1
    pr2: float = (vin - vout2) * iout2
    pout1: float = vout1 * iout1
    pout2: float = vout2 * iout2
    pout: float = pout1 + pout2
    pin: float = vin * -iin
    eta: float = pout / pin

    print(f"p1 = {pr1:.4g}W")
    print(f"p2 = {pr2:.4g}W")
    print(f"pout1 = {pout1:.4g}W")
    print(f"pout2 = {pout2:.4g}W")
    print(f"pout = {pout:.4g}W")
    print(f"pin = {pin:.4g}W")
    print(f"eta = {eta*100:.4g}%")

    return my_netlists_dict


# netlists_dict = part1(paths_dict, netlists_dict, vectors_dict)

#### Main
Run all the parts at once so the results are together.

Note that with each "part" function the netlist_dict is updated (returned) with the top netlist from each part added to the dictionary.

In [5]:
# Execute all parts
netlists_dict = part1(paths_dict, netlists_dict, vectors_dict)


--- Operating Point Results ---
in             12.000
out1           5.000
out1_com       0.000
out2           3.297
out2_com       0.000
vin#branch    −115.934m
vmeas1#branch  50.000m
vmeas2#branch  65.934m

-------------------------------

p1 = 0.35W
p2 = 0.5738W
pout1 = 0.25W
pout2 = 0.2174W
pout = 0.4674W
pin = 1.391W
eta = 33.59%
